# Interactive Germany 2033 Simulation

This notebook allows you to experiment with installed capacities for the Germany 2033 scenario and run the PyPSA simulation to observe the impact on failures and prices.

### Instructions:
1. **Edit Capacities**: Change the values in the "User Configuration" cell below.
2. **Run All**: Run all cells to execute the simulation.
3. **Analyze**: Scroll down to see plots of generation, prices, and any failures.

In [ ]:
# 1. Imports and Setup
import sys
import os
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from typing import List

# Add current directory to path to ensure imports work
sys.path.append(os.getcwd())

from common.constants.pypsa_params import GEN_UNITS_PYPSA_PARAMS
from common.fuel_sources import FuelSource, FuelNames, DummyFuelNames, set_fuel_sources_from_json, DUMMY_FUEL_SOURCES
from include.dataset_builder import GenerationUnitData, set_country_trigram, PypsaModel
from common.long_term_uc_io import set_simulation_run_id, get_json_fixed_params_file, HYDRO_KEY_COLUMNS, COLUMN_NAMES
from common.logger import deactivate_verbose_warnings
from common.constants.extract_eraa_data import ERAADatasetDescr
from common.plot_params import PlotParams
from common.constants.usage_params_json import EnvPhaseNames
from common.constants.prod_types import ProdTypeNames
from common.uc_run_params import UCRunParams
from include.dataset import Dataset
import include.dataset # Import module to patch it
from utils.df_utils import selec_in_df_based_on_list
from utils.eraa_data_reader import get_hydro_ror_generation, get_hydro_inflows, get_hydro_level_constraints
from common.long_term_uc_io import INPUT_ERAA_FOLDER
from utils.read import check_and_load_json_file, read_plot_params, read_given_phase_plot_params, read_solver_params
from utils.basic_utils import print_non_default
from common.constants.datadims import DataDimensions
from common.constants.optimisation import OPTIM_RESOL_STATUS

# --- PATCH: Fix for list.remove(x) error ---
def set_final_hydro_key_cols_patched(hydro_dt: str) -> List[str]:
    # Create a COPY of the list to avoid modifying the global constant
    key_cols = HYDRO_KEY_COLUMNS[hydro_dt].copy()
    
    # week and day idx columns have been removed when reading and processing
    for col in [COLUMN_NAMES.day, COLUMN_NAMES.week]:
        if col in key_cols:
            key_cols.remove(col)
            
    # not to be saved in df of a given country (with all fields of same value)
    zone_col = COLUMN_NAMES.zone
    if zone_col in key_cols:
        key_cols.remove(zone_col)
        
    # if data wo climatic year, but added to have uniform format hereafter
    if COLUMN_NAMES.climatic_year not in key_cols:
        key_cols.append(COLUMN_NAMES.climatic_year)
    return key_cols

# Apply the patch
include.dataset.set_final_hydro_key_cols = set_final_hydro_key_cols_patched
print("Applied patch for set_final_hydro_key_cols")
# -------------------------------------------

# --- PATCH: Fix for missing plots ---
# Disable plt.close so plots are not closed before being shown in the notebook.
# We do NOT redirect to plt.show() to avoid potential recursion if show() calls close().
plt.close = lambda *args, **kwargs: None
print("Applied patch for plt.close -> no-op")
# ------------------------------------

# Deactivate warnings
deactivate_verbose_warnings(deact_deprecation_warn=True)

Applied patch for set_final_hydro_key_cols
Applied patch for plt.close -> plt.show


In [2]:
# 2. User Configuration - EDIT THESE VALUES

# --- Simulation Parameters ---
YEAR = 2033
CLIMATIC_YEAR = 2016
START_MONTH = 1
START_DAY = 1
NUM_DAYS = 4  # Keep small for fast testing, increase for full analysis

# --- Installed Capacities (MW) ---
# Default values from ERAA 2023-2 for Germany 2033

# Conventional
GAS_CAPACITY = 35084.62
OIL_CAPACITY = 1044.83
OTHER_NON_RENEWABLE_CAPACITY = 5167.52

# Renewables
WIND_ONSHORE_CAPACITY = 133515.93
WIND_OFFSHORE_CAPACITY = 42185.0
SOLAR_PV_CAPACITY = 270886.85
OTHER_RENEWABLE_CAPACITY = 10355.84

# Storage
BATTERY_POWER = 11985.95
BATTERY_ENERGY = 23971.9

# Hydro (Usually fixed by geography, but editable)
PUMP_CLOSED_POWER_TURBINE = 7009.84
PUMP_CLOSED_POWER_PUMP = 7166.6
PUMP_CLOSED_ENERGY = 391579.78
PUMP_OPEN_POWER_TURBINE = 2144.1
PUMP_OPEN_POWER_PUMP = 1861.0
PUMP_OPEN_ENERGY = 471229.0
RESERVOIR_POWER = 819.0
RESERVOIR_ENERGY = 237217.03
ROR_POWER = 3933.9

# Phased out (Keep at 0 unless you want to test bringing them back)
HARD_COAL_CAPACITY = 0.0
LIGNITE_CAPACITY = 0.0

print("Configuration Loaded.")

Configuration Loaded.


In [3]:
# 3. Generator Definition Logic (Adapted to use variables above)

from typing import Dict, List, Union
GENERATOR_DICT_TYPE = Dict[str, Union[float, int, str]]
gps_coords = (13.4050, 52.5200)

def get_custom_generators(country_trigram: str, fuel_sources: Dict[str, FuelSource], wind_on_shore_cf_data: pd.DataFrame,
                   wind_off_shore_cf_data: pd.DataFrame, solar_pv_cf_data: pd.DataFrame,
                   hydro_ror_profile: pd.DataFrame = None,
                   hydro_inflows: dict = None,
                   hydro_level_constraints: dict = None) -> List[GENERATOR_DICT_TYPE]:
    
    generators = [
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_gas',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.gas,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: GAS_CAPACITY,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: fuel_sources[FuelNames.gas].primary_cost / 0.5,
            GEN_UNITS_PYPSA_PARAMS.efficiency: 0.5
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_oil',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.oil,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: OIL_CAPACITY,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: fuel_sources[FuelNames.oil].primary_cost / 0.4,
            GEN_UNITS_PYPSA_PARAMS.efficiency: 0.4
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_other_non_renewables',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.other_non_renewables,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: OTHER_NON_RENEWABLE_CAPACITY,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: fuel_sources[FuelNames.other_non_renewables].primary_cost / 0.4,
            GEN_UNITS_PYPSA_PARAMS.efficiency: 0.4
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_hard_coal',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.coal,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: HARD_COAL_CAPACITY,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: fuel_sources[FuelNames.coal].primary_cost / 0.4,
            GEN_UNITS_PYPSA_PARAMS.efficiency: 0.4
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_lignite',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.coal,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: LIGNITE_CAPACITY,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: fuel_sources[FuelNames.coal].primary_cost / 0.4,
            GEN_UNITS_PYPSA_PARAMS.efficiency: 0.4
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_wind_onshore',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.wind,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: WIND_ONSHORE_CAPACITY,
            GEN_UNITS_PYPSA_PARAMS.max_power_pu: wind_on_shore_cf_data['value'].values,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: fuel_sources[FuelNames.wind].primary_cost
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_wind_offshore',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.wind,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: WIND_OFFSHORE_CAPACITY,
            GEN_UNITS_PYPSA_PARAMS.max_power_pu: wind_off_shore_cf_data['value'].values,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: fuel_sources[FuelNames.wind].primary_cost
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_solar_pv',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.solar,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: SOLAR_PV_CAPACITY,
            GEN_UNITS_PYPSA_PARAMS.max_power_pu: solar_pv_cf_data['value'].values,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: fuel_sources[FuelNames.solar].primary_cost
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_other_renewables',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.other_renewables,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: OTHER_RENEWABLE_CAPACITY,
        },
        # Storage units
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_battery',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.battery,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: BATTERY_POWER,
            GEN_UNITS_PYPSA_PARAMS.max_hours: BATTERY_ENERGY / BATTERY_POWER if BATTERY_POWER > 0 else 0,
            GEN_UNITS_PYPSA_PARAMS.efficiency_store: 0.92,
            GEN_UNITS_PYPSA_PARAMS.efficiency_dispatch: 0.92,
            GEN_UNITS_PYPSA_PARAMS.cyclic_state_of_charge: True,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: 0.0
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_pump_closed',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.hydro_pump_closed,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: PUMP_CLOSED_POWER_TURBINE,
            GEN_UNITS_PYPSA_PARAMS.max_hours: PUMP_CLOSED_ENERGY / PUMP_CLOSED_POWER_TURBINE if PUMP_CLOSED_POWER_TURBINE > 0 else 0,
            GEN_UNITS_PYPSA_PARAMS.efficiency_store: 0.9,
            GEN_UNITS_PYPSA_PARAMS.efficiency_dispatch: 0.9,
            GEN_UNITS_PYPSA_PARAMS.cyclic_state_of_charge: True,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: 0.0,
            **({'state_of_charge_set': hydro_level_constraints['pump_closed_min']['value'].values * PUMP_CLOSED_ENERGY} 
               if hydro_level_constraints and hydro_level_constraints.get('pump_closed_min') is not None else {})
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_pump_open',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.hydro_pump_open,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: PUMP_OPEN_POWER_TURBINE,
            GEN_UNITS_PYPSA_PARAMS.max_hours: PUMP_OPEN_ENERGY / PUMP_OPEN_POWER_TURBINE if PUMP_OPEN_POWER_TURBINE > 0 else 0,
            GEN_UNITS_PYPSA_PARAMS.efficiency_store: 0.9,
            GEN_UNITS_PYPSA_PARAMS.efficiency_dispatch: 0.9,
            GEN_UNITS_PYPSA_PARAMS.cyclic_state_of_charge: False,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: 0.0,
            **({'inflow': hydro_inflows['pump_open']['value'].values} if hydro_inflows and hydro_inflows.get('pump_open') is not None else {}),
            **({'state_of_charge_set': hydro_level_constraints['pump_open_min']['value'].values * PUMP_OPEN_ENERGY} 
               if hydro_level_constraints and hydro_level_constraints.get('pump_open_min') is not None else {})
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_reservoir',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.hydro_reservoir,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: RESERVOIR_POWER,
            GEN_UNITS_PYPSA_PARAMS.max_hours: RESERVOIR_ENERGY / RESERVOIR_POWER if RESERVOIR_POWER > 0 else 0,
            GEN_UNITS_PYPSA_PARAMS.efficiency_dispatch: 0.9,
            GEN_UNITS_PYPSA_PARAMS.cyclic_state_of_charge: False,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: 0.0,
            **({'inflow': hydro_inflows['reservoir']['value'].values} if hydro_inflows and hydro_inflows.get('reservoir') is not None else {}),
            **({'state_of_charge_set': hydro_level_constraints['reservoir_min']['value'].values * RESERVOIR_ENERGY} 
               if hydro_level_constraints and hydro_level_constraints.get('reservoir_min') is not None else {})
        },
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_ror',
            GEN_UNITS_PYPSA_PARAMS.carrier: FuelNames.hydro_ror,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: ROR_POWER,
            GEN_UNITS_PYPSA_PARAMS.max_power_pu: (hydro_ror_profile['value'].values / ROR_POWER) if hydro_ror_profile is not None and ROR_POWER > 0 else 1.0,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: 0.0
        },
        # Fictive failure asset
        {
            GEN_UNITS_PYPSA_PARAMS.name: f'{country_trigram}_failure',
            GEN_UNITS_PYPSA_PARAMS.carrier: DummyFuelNames.ac,
            GEN_UNITS_PYPSA_PARAMS.nominal_power: 1e10,
            GEN_UNITS_PYPSA_PARAMS.marginal_cost: 1e5
        }
    ]
    return generators

def set_gen_as_list_of_gen_units_data(generators: List[GENERATOR_DICT_TYPE]) -> List[GenerationUnitData]:
    for elt_gen in generators:
        elt_gen['type'] = f'{elt_gen["carrier"]}_agg'
    return [GenerationUnitData(**elt_gen_dict) for elt_gen_dict in generators]

In [4]:
# 4. Run Simulation

# Setup Run ID
_temp_start = datetime(year=1900, month=START_MONTH, day=START_DAY)
_temp_end = _temp_start + timedelta(days=NUM_DAYS)
_timestamp = datetime.now().strftime('%Y%m%d_%H%M')
_simulation_run_id = f'INTERACTIVE_{YEAR}_cy{CLIMATIC_YEAR}_{START_MONTH:02d}{START_DAY:02d}-{_temp_end.month:02d}{_temp_end.day:02d}_{_timestamp}'
set_simulation_run_id(_simulation_run_id)

print(f'=== Starting Interactive Simulation ===')
print(f'Run ID: {_simulation_run_id}')

# Helper functions
def get_eraa_data_description() -> ERAADatasetDescr:
    json_fixed_params_file = get_json_fixed_params_file()
    json_params_fixed = check_and_load_json_file(json_file=json_fixed_params_file, file_descr='JSON fixed params')
    json_available_values_dummy = {'available_climatic_years': None, 'available_countries': None, 
                                   'available_aggreg_prod_types': None, 'available_intercos': None, 'available_target_years': None}
    json_params_fixed |= json_available_values_dummy
    return ERAADatasetDescr(**json_params_fixed)

phase_name = EnvPhaseNames.monozone_toy_uc_model
def get_plots_params() -> (PlotParams, PlotParams):
    per_dim_plot_params = read_plot_params()
    fig_style = read_given_phase_plot_params(phase_name=phase_name)
    return per_dim_plot_params[DataDimensions.agg_prod_type], per_dim_plot_params[DataDimensions.zone]

# Initialize Parameters
country = 'germany'
uc_period_start = datetime(year=1900, month=START_MONTH, day=START_DAY)
uc_period_end = uc_period_start + timedelta(days=NUM_DAYS)
agg_prod_types_selec = [ProdTypeNames.wind_onshore, ProdTypeNames.wind_offshore, ProdTypeNames.solar_pv]

uc_run_params = UCRunParams(selected_countries=[country], selected_target_year=YEAR,
                            selected_climatic_year=CLIMATIC_YEAR,
                            selected_prod_types={country: agg_prod_types_selec},
                            uc_period_start=uc_period_start,
                            uc_period_end=uc_period_end)

# Load Data
eraa_data_descr = get_eraa_data_description()
eraa_dataset = Dataset(agg_prod_types_with_cf_data=eraa_data_descr.agg_prod_types_with_cf_data)
eraa_dataset.get_countries_data(uc_run_params=uc_run_params, aggreg_prod_types_def=eraa_data_descr.aggreg_prod_types_def)
eraa_dataset.complete_data()

# Extract CF Data
prod_type_col = 'production_type_agg'
solar_pv_cf_data = {country: selec_in_df_based_on_list(df=eraa_dataset.agg_cf_data[country], selec_col=prod_type_col, selec_vals=[ProdTypeNames.solar_pv], rm_selec_col=True)}
wind_on_shore_cf_data = {country: selec_in_df_based_on_list(df=eraa_dataset.agg_cf_data[country], selec_col=prod_type_col, selec_vals=[ProdTypeNames.wind_onshore], rm_selec_col=True)}
wind_off_shore_cf_data = {country: selec_in_df_based_on_list(df=eraa_dataset.agg_cf_data[country], selec_col=prod_type_col, selec_vals=[ProdTypeNames.wind_offshore], rm_selec_col=True)}

# Load Hydro Data
hydro_folder = INPUT_ERAA_FOLDER
hydro_ror_profile = get_hydro_ror_generation(folder=hydro_folder, zone=country, climatic_year=CLIMATIC_YEAR, period_start=uc_run_params.uc_period_start, period_end=uc_run_params.uc_period_end)
hydro_inflows = get_hydro_inflows(folder=hydro_folder, zone=country, climatic_year=CLIMATIC_YEAR, period_start=uc_run_params.uc_period_start, period_end=uc_run_params.uc_period_end)
hydro_level_constraints = get_hydro_level_constraints(folder=hydro_folder, zone=country, climatic_year=CLIMATIC_YEAR, period_start=uc_run_params.uc_period_start, period_end=uc_run_params.uc_period_end)

# Build PyPSA Model
country_trigram = set_country_trigram(country=country)
pypsa_model = PypsaModel(name=f'Interactive {country_trigram} model')

date_idx = eraa_dataset.demand[uc_run_params.selected_countries[0]].index
horizon = pd.date_range(start=uc_run_params.uc_period_start.replace(year=uc_run_params.selected_target_year), end=uc_run_params.uc_period_end.replace(year=uc_run_params.selected_target_year), freq='h')
pypsa_model.init_pypsa_network(date_idx=date_idx, date_range=horizon)

coordinates = {country: gps_coords}
pypsa_model.add_gps_coordinates(countries_gps_coords=coordinates)

# Define Generators using CUSTOM function
fuel_sources = set_fuel_sources_from_json()
generators = get_custom_generators(country_trigram=country_trigram, fuel_sources=fuel_sources,
                            wind_on_shore_cf_data=wind_on_shore_cf_data[country],
                            wind_off_shore_cf_data=wind_off_shore_cf_data[country],
                            solar_pv_cf_data=solar_pv_cf_data[country],
                            hydro_ror_profile=hydro_ror_profile,
                            hydro_inflows=hydro_inflows,
                            hydro_level_constraints=hydro_level_constraints)
generation_units_data = set_gen_as_list_of_gen_units_data(generators=generators)
eraa_dataset.set_generation_units_data(gen_units_data={country: generation_units_data})

fuel_sources |= DUMMY_FUEL_SOURCES
pypsa_model.add_energy_carriers(fuel_sources=fuel_sources)
pypsa_model.add_per_bus_energy_carriers(fuel_sources=fuel_sources)
pypsa_model.add_generators(generators_data=eraa_dataset.generation_units_data)
pypsa_model.add_loads(demand=eraa_dataset.demand)

# Solve
solver_params = read_solver_params()
pypsa_model.set_optim_solver(solver_params=solver_params)
result = pypsa_model.optimize_network(year=uc_run_params.selected_target_year, n_countries=1,
                                      period_start=uc_run_params.uc_period_start, save_lp_file=False,
                                      toy_model_output=True, countries=[country])
print(f'Optimization Result: {result}')

-> replaced by {'value': 0}


=== Starting Interactive Simulation ===
Run ID: INTERACTIVE_2033_cy2016_0101-0105_20251119_1217


/Users/dh/development/electricsystemplanning-2025-europe_two/utils/eraa_data_reader.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/dh/development/electricsystemplanning-2025-europe_two/utils/eraa_data_reader.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-cj5kina4 has 4896 rows; 2208 cols; 7774 nonzeros
Coefficient ranges:
  Matrix  [9e-01, 1e+00]
  Cost    [2e+01, 1e+05]
  Bound   [0e+00, 0e+00]
  RHS     [4e+01, 1e+10]
Presolving model
476 rows, 1930 cols, 3036 nonzeros  0s
371 rows, 1403 cols, 2416 nonzeros  0s
Dependent equations search running on 356 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
356 rows, 1275 cols, 2168 nonzeros  0s
Presolve reductions: rows 356(-4540); columns 1275(-933); nonzeros 2168(-5606) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
        430     6.3785875652e+09 Pr: 0(0) 0s

Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-cj5kina4
Mode

In [ ]:
# 5. Analyze Results
optim_status = result[1]
if optim_status == OPTIM_RESOL_STATUS.optimal:
    objective_value = pypsa_model.get_opt_value(pypsa_resol_status=OPTIM_RESOL_STATUS.optimal)
    print(f'Total cost: {objective_value:,.2f} EUR')
    
    pypsa_model.get_prod_var_opt()
    pypsa_model.get_storage_vars_opt()
    pypsa_model.get_sde_dual_var_opt()

    plot_params_agg_pt, plot_params_zone = get_plots_params()
    
    print("\n--- Generation Mix ---")
    pypsa_model.plot_opt_prod_var(plot_params_agg_pt=plot_params_agg_pt, country=country,
                                  year=uc_run_params.selected_target_year,
                                  climatic_year=uc_run_params.selected_climatic_year,
                                  start_horizon=uc_run_params.uc_period_start,
                                  toy_model_output=True)
    
    print("\n--- Failures (Unmet Demand) ---")
    pypsa_model.plot_failure_at_opt(country=country, year=uc_run_params.selected_target_year,
                                    climatic_year=uc_run_params.selected_climatic_year,
                                    start_horizon=uc_run_params.uc_period_start,
                                    toy_model_output=True)
    
    print("\n--- Marginal Prices ---")
    pypsa_model.plot_marginal_price(plot_params_zone=plot_params_zone, year=uc_run_params.selected_target_year,
                                    climatic_year=uc_run_params.selected_climatic_year,
                                    start_horizon=uc_run_params.uc_period_start, toy_model_output=True,
                                    country=country)

    # --- Recommendation System ---
    print("\n=== Capacity Planning Recommendations ===")
    
    # Find failure generator
    try:
        failure_gen_name = [c for c in pypsa_model.network.generators_t.p.columns if 'failure' in c][0]
        failure_series = pypsa_model.network.generators_t.p[failure_gen_name]
        max_failure = failure_series.max()
        
        if max_failure > 1.0: # Threshold of 1 MW
            print(f"⚠️ SYSTEM FAILURE DETECTED: Unmet demand peaked at {max_failure:,.2f} MW.")
            print(f"👉 RECOMMENDATION: Increase GAS_CAPACITY by at least {max_failure:,.2f} MW to cover this peak.")
            print(f"   (Current GAS_CAPACITY: {GAS_CAPACITY:,.2f} MW -> Try {GAS_CAPACITY + max_failure:,.2f} MW)")
        else:
            print("✅ No significant unmet demand detected. The current capacity is sufficient for this scenario.")
    except Exception as e:
        print(f"Could not generate recommendations: {e}")

else:
    print("Optimization failed or was not optimal.")

Total cost: 6,378,587,565.16 EUR

--- Generation Mix ---

--- Failures (Unmet Demand) ---

--- Marginal Prices ---

--- Marginal Prices ---
